In [1]:
import cv2
import numpy as np
import random
import os

def rotate_image(image, angle):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_NEAREST)
    return rotated_image

def rotate_boxes(boxes, angle, center):
    radians = np.radians(angle)
    rotation_matrix = np.array([[np.cos(radians), -np.sin(radians)],
                                [np.sin(radians), np.cos(radians)]])

    # 添加一列用于存储类别信息
    boxes_with_cls = np.column_stack([boxes, np.zeros(boxes.shape[0])])

    # 将目标框坐标转换为相对于图像中心的坐标
    print(boxes_with_cls[:, 1:3],0.5)
    relative_boxes = boxes_with_cls[:, 1:3] - 0.5
    
    # 进行坐标旋转
    rotated_boxes = np.dot(relative_boxes, rotation_matrix.T)

    # 将坐标转换回原始坐标系
    rotated_boxes += 0.5

    rotated_boxes[rotated_boxes<0] = 0
    # print(rotated_boxes)
    # 更新目标框坐标
    boxes_with_cls[:, 1:3] = rotated_boxes
    
    return boxes_with_cls



def get_filename(path, name):
    tep = os.listdir(path)
    index = len(tep)
    while(1):
        if name + "_" + str(index)+".jpg" in tep or name + "_" + str(index)+".txt" in tep:
            index += 1
        else:
            return name + "_" + str(index)


def random_rotate(image, boxes, max_angle=6):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    angle = random.uniform(-max_angle, max_angle)
    rotated_image = rotate_image(image, angle)
    rotated_boxes = rotate_boxes(boxes, angle, center)
    return rotated_image, rotated_boxes

def rotate_augmentation(image_path,label_path,save_pic_path,save_label_path):
    # 读取图像和标签
    image = cv2.imread(image_path)
    boxes = np.loadtxt(label_path, dtype=float)
    if boxes.ndim == 1:
        boxes=boxes.reshape(1,-1)

    # 进行随机旋转增强
    rotated_image, rotate_boxes = random_rotate(image, boxes)
    f_name = get_filename(save_pic_path, "rotate")

    save_pic_path = os.path.join(save_pic_path,f_name+".jpg")
    save_label_path = os.path.join(save_label_path, f_name+".txt")

    cv2.imwrite(save_pic_path, rotated_image)
    with open(save_label_path,"w") as f:
        # 绘制修正后的检测框
        for boxes in rotate_boxes:  
            c,x,y,w,h,_ = boxes
            f.write("{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(int(c),x,y,w,h))

# """example"""
# rotate_augmentation('example.jpg','example.txt','./images/','./labels/')

[[0.410417 0.234259]
 [0.45651  0.232407]
 [0.678646 0.299074]
 [0.702604 0.300926]
 [0.653125 0.297222]
 [0.457031 0.605556]
 [0.432812 0.606944]] 0.5
./images/rotate_7.jpg ./labels/rotate_7.txt
[[8.00000000e+00 4.25479165e-01 2.29648307e-01 2.81250000e-02
  1.85190000e-02 0.00000000e+00]
 [8.00000000e+00 4.71603437e-01 2.30387261e-01 1.09370000e-02
  2.40740000e-02 0.00000000e+00]
 [0.00000000e+00 6.89645784e-01 3.09421598e-01 1.56250000e-02
  1.11110000e-02 0.00000000e+00]
 [0.00000000e+00 7.13462003e-01 3.12615872e-01 1.45830000e-02
  1.11110000e-02 0.00000000e+00]
 [0.00000000e+00 6.64269031e-01 3.06139565e-01 1.56250000e-02
  1.11110000e-02 0.00000000e+00]
 [1.40000000e+01 4.51172020e-01 6.02976854e-01 9.89600000e-03
  1.29630000e-02 0.00000000e+00]
 [1.00000000e+01 4.26913293e-01 6.03002814e-01 1.77080000e-02
  1.57410000e-02 0.00000000e+00]]


In [24]:
import numpy as np

# 给定的数组
array_a = np.array([[1, 2], [3, 4], [4, 5]])
array_b = np.array([[1, 1], [2, 2]])

# 对每一行执行点乘操作
result = np.dot(array_a, array_b.T)

# 输出结果
print(result)


[[ 3  6]
 [ 7 14]
 [ 9 18]]
